In [ ]:
# --- 1️⃣ Setup ---
!pip install -q ultralytics opencv-python torch matplotlib numpy pandas scipy

import cv2, os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from ultralytics import YOLO
from scipy.spatial.transform import Rotation as R

# --- 2️⃣ Prepare Synthetic Demo Data ---
!mkdir -p data/images data/poses results/detections

# download a demo image
!wget -q https://github.com/ultralytics/assets/releases/download/v0.0.0/bus.jpg -O data/images/object_1.jpg

# create dummy 6D pose labels
pose_data = pd.DataFrame({
    'image': ['object_1.jpg'],
    'x': [120.5], 'y': [85.3], 'z': [0.55],  # meters
    'roll': [10.0], 'pitch': [5.0], 'yaw': [45.0]  # degrees
})
pose_data.to_csv("data/poses/pose_labels.csv", index=False)
pose_data

# --- 3️⃣ YOLO-based Object Detection ---
model = YOLO('yolov8n.pt')  # pre-trained model
results = model.predict(source='data/images/object_1.jpg', save=True, project='results/detections')

# visualize detection
img = cv2.imread('data/images/object_1.jpg')
det_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(det_img)
plt.title("Detected Object (YOLOv8)")
plt.axis('off')
plt.show()

# --- 4️⃣ Estimate 6D Pose (Simplified Concept) ---
# we will simulate pose refinement based on bounding box center
x, y, z = 120.5, 85.3, 0.55
roll, pitch, yaw = 10, 5, 45
rotation_matrix = R.from_euler('xyz', [roll, pitch, yaw], degrees=True).as_matrix()
pose_matrix = np.hstack((rotation_matrix, np.array([[x], [y], [z]])))

print("Estimated 6D Pose Matrix:\n", pose_matrix)

# --- 5️⃣ Save Results ---
os.makedirs('results/visualization', exist_ok=True)
pose_data.to_csv('results/predicted_poses.csv', index=False)

# --- 6️⃣ Visualize Orientation Axes ---
def draw_axes(img, origin, R_mat, length=50):
    cx, cy = int(origin[0]), int(origin[1])
    axes = np.array([[length,0,0], [0,length,0], [0,0,length]], dtype=float)
    colors = [(255,0,0), (0,255,0), (0,0,255)]
    for vec, col in zip(axes, colors):
        pt = (int(cx+vec[0]), int(cy+vec[1]))
        cv2.line(img, (cx,cy), pt, col, 2)
    return img

img_pose = draw_axes(img.copy(), (x, y), rotation_matrix)
plt.imshow(cv2.cvtColor(img_pose, cv2.COLOR_BGR2RGB))
plt.title("Estimated Pose Visualization")
plt.axis('off')
plt.show()
